In [2]:
import os
import subprocess 
import json

In [8]:
PWD:str = "/home/vincentwilde/Documents/thesis_1/Testymolo/testymolo/datamolo/scr/workinprogress/annotate"
reference_path:str = "./in/reference.fasta"
out_path:str = "./data/FASTA"

# DB

change header \
Export as Profile

In [4]:
####  VAZy 2  ####
vazy2_path:str = "./../../../../testymolo/media/data/"
vazy2_list:list = [ 'Modulo', 'Profile', 'Organism', 'PolyProtein', 'Protein', 'Subseq', ]
vazy2:dict= {}

def Load(param:str) -> list:
    fileName:str = os.path.join(vazy2_path, param+'.temp.json')
    with open(fileName) as handle:
        return json.load(handle)

for tabName in vazy2_list:
    vazy2[tabName] = Load(tabName)

In [5]:
####  trace  ####
trace:dict = {}

for profile in vazy2['Profile']:
    k = profile['id']
    modId = profile['modulo']
    if modId.startswith('F'):
        trace[modId] =  []
        #filter
        for subSeq in vazy2['Subseq']:
            if subSeq['profile'] == k:
                trace[modId].append((subSeq['id'], subSeq['origin'], subSeq['profile'], subSeq['start'], subSeq['end']))

with open("./data/trace.json", 'w') as out :
    out.write(json.dumps(trace))

In [6]:
####  Export  &  change header  ####

for mod, SubSeqs in trace.items():
    fasta:list = []
    for ss in SubSeqs:
        #filter
        for item in vazy2['Protein']:
            if item['id'] == ss[1]:
                header:str = '>'+str(ss[2])+':'+str(item['id'])+f":[{ss[-2]}-{ss[-1]}]\n"  # change header
                sequence:str = item['sequence'][ss[-2]-1:ss[-1]-1]+'\n'
                fasta.append(header+sequence)
                break
    #endfor

    with open(f"./fasta/{mod}.fasta", 'w') as out:  # Export
        out.write(''.join(fasta))

*profiles -> DB

In [7]:
def profile2DB():
    p_profiles:str = "./fasta"
    fn_profiles:list = os.listdir(p_profiles)
    out_path:str = "./data/FASTA"
    #print(os.path.join(p_profiles, fn_profiles[0]))
    lst_profiles:str = ' '.join([ os.path.join(p_profiles, fn_profiles[i]) for i in range(len(fn_profiles))])
    command = f"mmseqs createdb {lst_profiles} {out_path} -v 0"
    subprocess.Popen(command.split(' '), stdout=subprocess.DEVNULL)

profile2DB()


# Align

In [9]:
command:str = f"mmseqs easy-search {reference_path} {out_path} ./out/out.aln.fasta ./tmp -v 0"
print(command)
subprocess.Popen(command.split(' '), stdout=subprocess.DEVNULL)

mmseqs easy-search ./in/reference.fasta ./data/FASTA ./out/out.aln.fasta ./tmp -v 0


<Popen: returncode: None args: ['mmseqs', 'easy-search', './in/reference.fas...>

SARS-CoV-2 (TaxId) : 2697049 \
pp1ab (id) : 270

# Muscle

In [32]:
from Bio.Align.Applications import MuscleCommandline
import subprocess 

In [33]:
from Bio import SeqIO

sequences:list = []



In [49]:
## align profiles 
with open("./in/reference.fasta") as handle:
    sequences = SeqIO.parse(handle, format='fasta')
    for i, seq in enumerate(sequences):

        for file in os.listdir("./fasta"):
            profile_path:str = os.path.join("./fasta", file)

            #profile
            profile = ""
            with open(profile_path) as handle:
                profile = handle.read()

            #inFile
            inFile = f"./in/sequences/{str(i)}_{file}"
            with open(inFile, 'w') as handle:
                handle.write(f">{seq.id}\n")
                handle.write(f"{seq.seq}\n")
                handle.write(profile)

            out:str = f"./out/muscle/{str(i)}_{file}.clwstrict"
            command:str = f"muscle -in {inFile} -out {out} -clwstrict -quiet"
            #print(command)  # adding "-quiet" option ... doen't work in v3.8 ?! (but does in v5.)
            subprocess.Popen(command.split(' '), stdout=subprocess.DEVNULL)  # command must be a list of words

            break
        break

        
    

In [ ]:
data:list = []

out_dir_path:str = "./out/muscle"
for outFile in os.listdir(out_dir_path):
    outPath:str = os.path.join(out_dir_path, outFile)
    #